# GETTING AND PREPARING THE DATA

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("maruti_suzuki/MARUTI.NS.csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("maruti_suzuki/maruti_suzuki_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
sent_model = tf.keras.models.load_model("final_bert")
# removing duplicates from the data
final_posts.drop_duplicates(subset=['Messages'])
# calculating the sentiments score
sentiments = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        _,msg,time = j
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)
# getting indices where sentiments score is 0
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
# removing all the zero values indices
sentiments = np.delete(sentiments,zero_index)
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
# removing the added helper column
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

In [9]:
k = int(np.round(len(final_x_zeros)*0.8))
k

50

# BUILDING THE MODEL

In [10]:
#building the blocks
class blocks(tf.keras.layers.Layer):
    def __init__(self,
                 number_neurons:int,
                 number_layers:int,
                 window_size:int,
                 horizon_size:int,
                 theta_size:int,
                 **kwargs): #**kwargs takes care of the arguements of the parent class(input_size)
        super().__init__(**kwargs)
        self.hidden_neurons = number_neurons
        self.hidden_layers = number_layers
        self.output_size = horizon_size
        self.input_size = window_size
        self.theta_size = theta_size
        
        self.fc_layers = [tf.keras.layers.Dense(self.hidden_neurons, activation = "relu") for _ in range(self.hidden_layers)]
        self.theta_layer = tf.keras.layers.Dense(self.theta_size, name = "theta")
    def call(self, inputs):
        x = inputs 
        for i in self.fc_layers:
            x = i(x)
        outputs = self.theta_layer(x)
        backcasts, forecasts = outputs[:,:self.input_size], outputs[:,-int(self.output_size):]
        return backcasts, forecasts
# testing the block layer
trial_data = tf.expand_dims(tf.range(7),axis=0).numpy()
block = blocks(number_neurons = 512,
                 number_layers = 4,
                 window_size = 7,
                 horizon_size = 1,
                 theta_size = 8, # theta_size = window_size(backcast) + horizon_size(forecast)
              )
backcasts, forecasts = block(trial_data)
backcasts, forecasts

(<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
 array([[ 0.32581076, -0.11805074,  0.12559159, -0.12523536,  0.02738374,
         -0.07395406,  0.14072429]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.38247153]], dtype=float32)>)

# BUILDING, COMPILING AND FITTING THE MODEL

In [11]:
# setting up the hyper parameters
number_neurons = 512
number_layers = 4
epochs = 5000
number_stacks = 30
input_size = 71
output_size = 1
theta_size = 72

In [12]:
tf.random.set_seed(42)
block1 = blocks(number_neurons,
                 number_layers,
                 input_size,
                 output_size,
                 theta_size)
stack_inputs = tf.keras.layers.Input(shape = (input_size))
backcasts,forecasts = block1(stack_inputs)
residual = tf.keras.layers.subtract([stack_inputs,backcasts])
for i,_ in enumerate(range(number_stacks-1)):
    backcasts, block_forecasts = blocks(number_neurons,
                                 number_layers,
                                 input_size,
                                 output_size,
                                 theta_size)(residual)
    residual = tf.keras.layers.subtract([residual, backcasts])
    forecasts = tf.keras.layers.add([forecasts, block_forecasts])
model_nbeats = tf.keras.models.Model(inputs = stack_inputs, outputs = forecasts)
# compiling the model
model_nbeats.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])
# fitting the model
history = model_nbeats.fit(final_x_zeros[:k],final_y_zeros[:k],
            epochs = epochs,
            validation_data=(final_x_zeros[k:],final_y_zeros[k:]), 
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 1/5000
2/2 [==============================] - 6s 1s/step - loss: 137669.6250 - mae: 137669.6250 - mse: 46847045632.0000 - val_loss: 68247.0859 - val_mae: 68247.0859 - val_mse: 4659890688.0000 - lr: 0.0010
Epoch 2/5000
2/2 [==============================] - 0s 264ms/step - loss: 42405.3516 - mae: 42405.3516 - mse: 2552830976.0000 - val_loss: 28412.8691 - val_mae: 28412.8691 - val_mse: 807964928.0000 - lr: 0.0010
Epoch 3/5000
2/2 [==============================] - 0s 230ms/step - loss: 24994.5547 - mae: 24994.5547 - mse: 630059840.0000 - val_loss: 3130.4424 - val_mae: 3130.4424 - val_mse: 9847427.0000 - lr: 0.0010
Epoch 4/5000
2/2 [==============================] - 0s 221ms/step - loss: 7766.5337 - mae: 7766.5337 - mse: 97890752.0000 - val_loss: 175.2641 - val_mae: 175.2641 - val_mse: 51506.6562 - lr: 0.0010
Epoch 5/5000
2/2 [==============================] - 0s 105ms/step - loss: 632.8467 - mae: 632.8466 - mse: 585771.0000 - val_loss: 14881.7656 - val_mae: 14881.7656 - val_mse: 22

2/2 [==============================] - 0s 113ms/step - loss: 198.2799 - mae: 198.2799 - mse: 59695.5312 - val_loss: 195.6655 - val_mae: 195.6655 - val_mse: 44330.6680 - lr: 0.0010
Epoch 43/5000
2/2 [==============================] - 0s 104ms/step - loss: 366.1038 - mae: 366.1038 - mse: 214295.2969 - val_loss: 508.0721 - val_mae: 508.0721 - val_mse: 290488.0312 - lr: 0.0010
Epoch 44/5000
2/2 [==============================] - 0s 101ms/step - loss: 461.3286 - mae: 461.3286 - mse: 268558.7500 - val_loss: 469.7678 - val_mae: 469.7678 - val_mse: 249540.2031 - lr: 0.0010
Epoch 45/5000
2/2 [==============================] - 0s 110ms/step - loss: 424.2224 - mae: 424.2224 - mse: 239203.6875 - val_loss: 471.1277 - val_mae: 471.1277 - val_mse: 255555.2656 - lr: 0.0010
Epoch 46/5000
2/2 [==============================] - 0s 99ms/step - loss: 284.7104 - mae: 284.7104 - mse: 102776.6328 - val_loss: 212.2283 - val_mae: 212.2283 - val_mse: 78134.9062 - lr: 0.0010
Epoch 47/5000
2/2 [===================

2/2 [==============================] - 0s 116ms/step - loss: 243.1678 - mae: 243.1678 - mse: 101446.2031 - val_loss: 122.0233 - val_mae: 122.0233 - val_mse: 34799.6133 - lr: 0.0010
Epoch 85/5000
2/2 [==============================] - 0s 100ms/step - loss: 257.9082 - mae: 257.9082 - mse: 99248.4219 - val_loss: 272.7048 - val_mae: 272.7048 - val_mse: 99248.8203 - lr: 0.0010
Epoch 86/5000
2/2 [==============================] - 0s 103ms/step - loss: 304.5580 - mae: 304.5580 - mse: 146124.4531 - val_loss: 412.6646 - val_mae: 412.6646 - val_mse: 201715.5469 - lr: 0.0010
Epoch 87/5000
2/2 [==============================] - 0s 98ms/step - loss: 347.3273 - mae: 347.3273 - mse: 182745.8438 - val_loss: 673.1314 - val_mae: 673.1314 - val_mse: 485780.2812 - lr: 0.0010
Epoch 88/5000
2/2 [==============================] - 0s 97ms/step - loss: 403.5752 - mae: 403.5752 - mse: 214458.0625 - val_loss: 295.7567 - val_mae: 295.7567 - val_mse: 100625.7422 - lr: 0.0010
Epoch 89/5000
2/2 [====================

2/2 [==============================] - 0s 122ms/step - loss: 167.3217 - mae: 167.3217 - mse: 53716.8867 - val_loss: 132.3182 - val_mae: 132.3182 - val_mse: 26382.1484 - lr: 0.0010
Epoch 127/5000
2/2 [==============================] - 0s 121ms/step - loss: 136.9390 - mae: 136.9390 - mse: 28573.1445 - val_loss: 474.3162 - val_mae: 474.3162 - val_mse: 258207.2031 - lr: 0.0010
Epoch 128/5000
2/2 [==============================] - 0s 99ms/step - loss: 299.9478 - mae: 299.9478 - mse: 120851.7812 - val_loss: 314.1314 - val_mae: 314.1314 - val_mse: 115399.2500 - lr: 0.0010
Epoch 129/5000
2/2 [==============================] - 0s 116ms/step - loss: 293.1681 - mae: 293.1681 - mse: 116964.0703 - val_loss: 395.5126 - val_mae: 395.5126 - val_mse: 184674.4375 - lr: 0.0010
Epoch 130/5000
2/2 [==============================] - 0s 98ms/step - loss: 287.5812 - mae: 287.5812 - mse: 106939.1797 - val_loss: 247.8268 - val_mae: 247.8268 - val_mse: 67566.3906 - lr: 0.0010
Epoch 131/5000
2/2 [================

Epoch 167/5000
2/2 [==============================] - 0s 94ms/step - loss: 104.8496 - mae: 104.8496 - mse: 22401.3301 - val_loss: 115.2359 - val_mae: 115.2359 - val_mse: 33441.7422 - lr: 1.0000e-04
Epoch 168/5000
2/2 [==============================] - 0s 99ms/step - loss: 143.7613 - mae: 143.7613 - mse: 29059.7832 - val_loss: 115.4897 - val_mae: 115.4897 - val_mse: 28762.0488 - lr: 1.0000e-04
Epoch 169/5000
2/2 [==============================] - 0s 111ms/step - loss: 156.9179 - mae: 156.9179 - mse: 33031.1250 - val_loss: 114.5549 - val_mae: 114.5549 - val_mse: 33669.2383 - lr: 1.0000e-04
Epoch 170/5000
2/2 [==============================] - 0s 95ms/step - loss: 102.6421 - mae: 102.6421 - mse: 17018.3457 - val_loss: 188.8533 - val_mae: 188.8533 - val_mse: 62941.8789 - lr: 1.0000e-04
Epoch 171/5000
2/2 [==============================] - 0s 101ms/step - loss: 116.8224 - mae: 116.8224 - mse: 25035.8145 - val_loss: 201.0859 - val_mae: 201.0859 - val_mse: 68222.1250 - lr: 1.0000e-04
Epoch 17

Epoch 209/5000
2/2 [==============================] - 0s 97ms/step - loss: 41.1248 - mae: 41.1248 - mse: 6689.3149 - val_loss: 147.7434 - val_mae: 147.7434 - val_mse: 48084.1719 - lr: 1.0000e-04
Epoch 210/5000
2/2 [==============================] - 0s 99ms/step - loss: 36.7305 - mae: 36.7305 - mse: 6701.0796 - val_loss: 147.8553 - val_mae: 147.8553 - val_mse: 47835.7852 - lr: 1.0000e-04
Epoch 211/5000
2/2 [==============================] - 0s 96ms/step - loss: 41.2244 - mae: 41.2244 - mse: 5858.8662 - val_loss: 135.2471 - val_mae: 135.2471 - val_mse: 43411.6250 - lr: 1.0000e-04
Epoch 212/5000
2/2 [==============================] - 0s 96ms/step - loss: 41.2661 - mae: 41.2661 - mse: 5489.9448 - val_loss: 143.5891 - val_mae: 143.5891 - val_mse: 46962.2617 - lr: 1.0000e-04
Epoch 213/5000
2/2 [==============================] - 0s 121ms/step - loss: 37.4427 - mae: 37.4427 - mse: 6345.6226 - val_loss: 157.8035 - val_mae: 157.8035 - val_mse: 51254.3594 - lr: 1.0000e-04
Epoch 214/5000
2/2 [====

Epoch 251/5000
2/2 [==============================] - 0s 96ms/step - loss: 38.9985 - mae: 38.9985 - mse: 3807.1023 - val_loss: 123.0228 - val_mae: 123.0228 - val_mse: 35436.1289 - lr: 1.0000e-04
Epoch 252/5000
2/2 [==============================] - 0s 98ms/step - loss: 36.7699 - mae: 36.7699 - mse: 3042.0056 - val_loss: 132.6228 - val_mae: 132.6228 - val_mse: 40349.6211 - lr: 1.0000e-04
Epoch 253/5000
2/2 [==============================] - 0s 104ms/step - loss: 25.5729 - mae: 25.5729 - mse: 2996.6895 - val_loss: 140.6244 - val_mae: 140.6244 - val_mse: 43882.2188 - lr: 1.0000e-04
Epoch 254/5000
2/2 [==============================] - 0s 112ms/step - loss: 29.4613 - mae: 29.4613 - mse: 3534.0469 - val_loss: 123.1637 - val_mae: 123.1637 - val_mse: 37168.6836 - lr: 1.0000e-04
Epoch 255/5000
2/2 [==============================] - 0s 98ms/step - loss: 35.1016 - mae: 35.1016 - mse: 3144.4360 - val_loss: 142.4595 - val_mae: 142.4595 - val_mse: 45882.9570 - lr: 1.0000e-04
Epoch 256/5000
2/2 [===

# EVALUATING THE MODEL

In [13]:
model_nbeats.evaluate(final_x_zeros[k:], final_y_zeros[k:])

1/1 [==============================] - 0s 35ms/step - loss: 100.4456 - mae: 100.4456 - mse: 23181.2988


[100.445556640625, 100.445556640625, 23181.298828125]

In [14]:
model_nbeats.save("nbeats_maruti suzuki")

INFO:tensorflow:Assets written to: nbeats_maruti suzuki\assets


INFO:tensorflow:Assets written to: nbeats_maruti suzuki\assets
c:\users\harshvardhan bhosale\onedrive\desktop\webscrapping\ws_env\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
c:\users\harshvardhan bhosale\onedrive\desktop\webscrapping\ws_env\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [15]:
len(final_x_zeros[k:])

12